# **Problem Statement: Identifying Employees with Excessive Meeting Hours**

## **Objective:**
The goal is to identify employees who consistently spend a significant amount of time in meetings (more than 20 hours per week) across multiple weeks. This helps in:
- Recognizing overburdened employees.
- Analyzing meeting-heavy departments.
- Improving workload distribution.

### **Key Metrics:**
- **Eligibility Threshold:** A week is considered "meeting-heavy" if an employee spends >20 hours in meetings (assuming a 40-hour workweek).
- **Consistency Check:** Only employees with **more than one** such week are reported.
- **Output:** Employee details sorted by the number of heavy weeks (descending) and then by name.

---

# **Example Test Sets**

## **Test Set 1: Basic Case**

### **Input Tables:**

#### **`employee` Table:**
| employee_id | employee_name | department  |
|-------------|---------------|-------------|
| 101         | Alice Smith   | Engineering |
| 102         | Bob Johnson   | Marketing   |
| 103         | Carol Lee     | Engineering |
| 104         | David Brown   | HR          |

#### **`meeting` Table:**
| employee_id | meeting_date | duration_hours |
|-------------|--------------|----------------|
| 101         | 2023-10-02   | 12             | (Week 40, Monday)
| 101         | 2023-10-04   | 10             |
| 101         | 2023-10-09   | 15             | (Week 41, Monday)
| 101         | 2023-10-11   | 8              |
| 102         | 2023-10-02   | 25             | (Week 40, Monday)
| 102         | 2023-10-16   | 22             | (Week 42, Monday)
| 103         | 2023-10-03   | 18             | (Week 40, Tuesday)
| 104         | 2023-10-05   | 5              | (Week 40, Thursday)

### **Expected Output:**
| employee_id | employee_name | department  | meeting_heavy_weeks |
|-------------|---------------|-------------|----------------------|
| 101         | Alice Smith   | Engineering | 2                    |
| 102         | Bob Johnson   | Marketing   | 2                    |

**Explanation:**
- **Alice Smith** has **2 heavy weeks** (Week 40: 22h, Week 41: 23h).
- **Bob Johnson** has **2 heavy weeks** (Week 40: 25h, Week 42: 22h).
- **Carol Lee** has only **1 heavy week** (Week 40: 18h → excluded due to `HAVING COUNT(*) > 1`).
- **David Brown** does not meet the threshold (Week 40: 5h → too low).

---

## **Test Set 2: Edge Cases (Single Week, No Heavy Weeks)**

### **Input Tables:**

#### **`employee` Table:**
| employee_id | employee_name | department  |
|-------------|---------------|-------------|
| 201         | Emma Davis    | Sales       |
| 202         | Frank Wilson  | Finance     |
| 203         | Grace Taylor  | Engineering |

#### **`meeting` Table:**
| employee_id | meeting_date | duration_hours |
|-------------|--------------|----------------|
| 201         | 2023-11-06   | 21             | (Week 45, Monday)
| 201         | 2023-11-08   | 19             |
| 202         | 2023-11-13   | 10             | (Week 46, Monday)
| 203         | 2023-11-06   | 25             | (Week 45, Monday)
| 203         | 2023-11-20   | 30             | (Week 47, Monday)

### **Expected Output:**
| employee_id | employee_name | department  | meeting_heavy_weeks |
|-------------|---------------|-------------|----------------------|
| 203         | Grace Taylor  | Engineering | 2                    |

**Explanation:**
- **Grace Taylor** has **2 heavy weeks** (Week 45: 25h, Week 47: 30h).
- **Emma Davis** has **only 1 heavy week** (Week 45: 40h total → excluded due to `HAVING COUNT(*) > 1`).
- **Frank Wilson** does not meet the threshold (Week 46: 10h → too low).

---

## **Key Takeaways from Test Cases:**
1. **Only employees with >1 heavy week** are included.
2. **A week is "heavy" if total meeting hours >20** (i.e., `SUM(duration_hours) > 20`).
3. **Weeks start on Monday** (ISO standard).
4. **Results are sorted** by `meeting_heavy_weeks` (descending) and then by `employee_name`.

This ensures we only flag employees who **consistently** spend excessive time in meetings, not just those with a single busy week.

In [ ]:
WITH weekly_hours AS (
    SELECT 
        employee_id,
        EXTRACT(WEEK FROM meeting_date) AS iso_week_number,  -- ISO week (Monday start)
        date_trunc('week', meeting_date)::date AS week_start_date,  -- Shows the Monday date
        SUM(duration_hours) AS total_hours,
        CASE WHEN SUM(duration_hours)/40 > 0.5 THEN 'Y' ELSE 'N' END AS eligibility
    FROM meeting
    GROUP BY employee_id, EXTRACT(WEEK FROM meeting_date), date_trunc('week', meeting_date)
),

employee_heavy_weeks AS (
    SELECT 
        c.employee_id,
        e.employee_name, 
        e.department, 
        COUNT(*) AS meeting_heavy_weeks
    FROM weekly_hours c
    LEFT JOIN employee e ON c.employee_id = e.employee_id
    WHERE c.eligibility = 'Y'
    GROUP BY c.employee_id, e.employee_name, e.department
    HAVING COUNT(*) > 1
)

SELECT 
    employee_id,
    employee_name, 
    department, 
    meeting_heavy_weeks
FROM employee_heavy_weeks
ORDER BY meeting_heavy_weeks DESC, employee_name;

---

## ✅ What is **ISO Week**?

**ISO Week** is a standardized way to define **weeks in a year** based on the **ISO 8601** international standard.

---

### 🧾 Key Characteristics of ISO Week:

| Feature               | Description                                                                 |
| --------------------- | --------------------------------------------------------------------------- |
| 📅 Week starts on     | **Monday**                                                                  |
| 🔢 Week numbering     | From **1 to 52/53** (not 0-based)                                           |
| 🗓 Week 1 definition  | The **week containing the first Thursday** of the year                      |
| 🔁 Fixed length?      | No, can have either **52 or 53 weeks** depending on the year                |
| 🧮 Affects edge dates | Some **Dec 29–31** dates may belong to **Week 1 of next year**, not current |

---

### 📊 Example (2024):

* Jan 1, 2024 → Monday → ISO Week **1**
* Dec 30, 2024 → Monday → ISO Week **1** **(of 2025)**

> Because Dec 30, 2024 belongs to the week that contains **Thursday, Jan 2, 2025**.

---

## ✅ What is **ISO 8601**?

**ISO 8601** is an **international standard** defined by the **International Organization for Standardization (ISO)** for representing **date and time formats**.

---

### 📚 Full Name:

> **ISO 8601: Data elements and interchange formats — Information interchange — Representation of dates and times**

---

### 🧾 Key Features of ISO 8601:

| Format           | Description                         | Example                       |
| ---------------- | ----------------------------------- | ----------------------------- |
| 📆 Date format   | `YYYY-MM-DD`                        | `2024-08-04`                  |
| ⏰ Time format    | `HH:MM:SS` (24-hour)                | `14:35:00`                    |
| 📆📆 DateTime    | Combined with `T` in between        | `2024-08-04T14:35:00`         |
| 🌍 Timezone      | With offset: `Z` (UTC) or `+06:00`  | `2024-08-04T14:35:00+06:00`   |
| 📅 ISO Week date | `YYYY-Www-D` (Year-Week-Day format) | `2024-W01-1` = Monday, Week 1 |

---

### 🧠 Why is ISO 8601 important?

It solves problems with:

* Ambiguous dates (e.g., `01-04-2024` → Jan 4 or Apr 1?)
* Week-based reports
* International data sharing (same format globally)
* Standardized week-start for analytics

---

### 📊 ISO Week Date Example:

```
ISO Week:      2024-W01
ISO Week Date: 2024-W01-1 → Monday
               2024-W01-7 → Sunday
```

You can even use this in PostgreSQL:

```sql
SELECT to_char('2024-01-01'::date, 'IYYY-"W"IW-D');
-- Output: 2024-W01-1
```

---

## 🧪 PostgreSQL & ISO 8601

PostgreSQL uses **ISO 8601** behavior by default for:

* `EXTRACT(WEEK FROM date)` → ISO Week Number
* `to_char(..., 'IYYY')` or `'IW'` → ISO Year/Week

---

## ✅ Summary

| Concept    | What It Means                                                                 |
| ---------- | ----------------------------------------------------------------------------- |
| ISO Week   | Week system defined by ISO 8601 where week starts on Monday                   |
| ISO 8601   | Global standard for dates, times, week formats                                |
| PostgreSQL | Adopts ISO 8601 week system in `EXTRACT(WEEK ...)`, `date_trunc('week', ...)` |

---


Both `EXTRACT(WEEK FROM ...)` and `date_trunc('week', ...)` are widely used in **time-series data**, **reporting dashboards**, and **aggregated analytics** in PostgreSQL. Let’s break them down with full **details**, **examples**, and **real use cases** so you understand not just what they do, but how and *why* they are used in real-world scenarios.

---

## ✅ 1. `EXTRACT(WEEK FROM meeting_date)` → **Get ISO Week Number**

### 🔍 What it does:

This function returns the **week number of the year** from a date (1–53), following **ISO 8601**:

* Week starts on **Monday**.
* Week 1 is the one with the **first Thursday** of the year.

> 💡 Not to be confused with `EXTRACT(ISODOW ...)`, which extracts **ISO Day Of Week** (1 = Monday, ..., 7 = Sunday).

---

### 🧠 Syntax:

```sql
EXTRACT(WEEK FROM date_column)
```

---

### 📊 Example Table: `team_meetings`

| meeting\_id | meeting\_date |
| ----------- | ------------- |
| 1           | 2024-01-01    |
| 2           | 2024-01-07    |
| 3           | 2024-01-08    |
| 4           | 2024-12-30    |

---

### 🧪 Example Query:

```sql
SELECT 
  meeting_id,
  meeting_date,
  EXTRACT(WEEK FROM meeting_date) AS iso_week_number
FROM team_meetings;
```

---

### 🧾 Result:

| meeting\_id | meeting\_date | iso\_week\_number |
| ----------- | ------------- | ----------------- |
| 1           | 2024-01-01    | 1                 |
| 2           | 2024-01-07    | 1                 |
| 3           | 2024-01-08    | 2                 |
| 4           | 2024-12-30    | 1                 |

✅ Notice how:

* Jan 1 and Jan 7 fall into **ISO Week 1**
* Jan 8 is the **start of ISO Week 2**
* Dec 30 again falls into **Week 1 of next year** per ISO standard

---

### 🛠 Use Case:

**Weekly KPI Reporting**:
Suppose you track team productivity and need to group by **ISO week**:

```sql
SELECT 
  EXTRACT(WEEK FROM meeting_date) AS iso_week,
  COUNT(*) AS total_meetings
FROM team_meetings
GROUP BY iso_week
ORDER BY iso_week;
```

---

## ✅ 2. `date_trunc('week', meeting_date)::date` → **Get Monday of That Week**

### 🔍 What it does:

It **truncates a date to the start of the week**, which is **Monday** in PostgreSQL's default configuration.

> 🗓 This returns the **Monday of that week**, helping you **group** or **label** weeks properly.

---

### 🧠 Syntax:

```sql
date_trunc('week', date_column)::date
```

> 🧠 `::date` is used to strip off the time part (which defaults to midnight after `date_trunc` anyway, but keeps it clean).

---

### 🧪 Example Query:

```sql
SELECT 
  meeting_id,
  meeting_date,
  date_trunc('week', meeting_date)::date AS week_start_date
FROM team_meetings;
```

---

### 🧾 Result:

| meeting\_id | meeting\_date | week\_start\_date |
| ----------- | ------------- | ----------------- |
| 1           | 2024-01-01    | 2024-01-01 (Mon)  |
| 2           | 2024-01-07    | 2024-01-01        |
| 3           | 2024-01-08    | 2024-01-08        |
| 4           | 2024-12-30    | 2024-12-30        |

---

### 🛠 Use Case:

**Weekly Revenue Chart**:
If you're charting revenue week-by-week, you can group all dates by their corresponding **Monday**:

```sql
SELECT 
  date_trunc('week', sale_date)::date AS week_start,
  SUM(amount) AS weekly_sales
FROM sales
GROUP BY week_start
ORDER BY week_start;
```

---

## 🧩 Putting It Together

You’ll often **combine** both to get week numbers **and** start dates in one dataset:

```sql
SELECT
  meeting_id,
  meeting_date,
  EXTRACT(WEEK FROM meeting_date) AS iso_week_number,
  date_trunc('week', meeting_date)::date AS week_start_date
FROM team_meetings;
```

---

## 🧠 Real-Life Use Case in Data Engineering

Imagine you're building a **weekly dashboard** in a company to monitor:

* Weekly active users
* Weekly new prescriptions
* Weekly diagnostic reports

You’d build materialized views or temp tables that group by:

* `EXTRACT(WEEK FROM report_date)`
* `date_trunc('week', report_date)::date`

So you can chart like:

```sql
SELECT
  date_trunc('week', report_date)::date AS week_start,
  COUNT(DISTINCT user_id) AS weekly_active_users
FROM user_reports
GROUP BY week_start
ORDER BY week_start;
```

---